In [ ]:
!nvidia-smi

Fri Apr  5 12:43:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0
!pip3 install -q -U rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=b00665bf108ad88b5c82e3b26da9a197739371badf44c9adce4b2dde0d5f5752
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [ ]:
import torch
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
dataset = load_dataset("rPucs/TripletDollyQA-0.1", split="train")

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
model_id = "rPucs/gemma2b-it-tripletdolly-0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quent_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          add_eos_token=True,
                                          padding_side="left"
                                          )

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

In [ ]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template="""
  <start_of_turn>user
  Below is an instruction or question that describes a task or requires an answer. Write a response that appropriately completes the request.
  {query} <end_of_turn>\n
  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded).split(query)[1]

In [ ]:
print(get_completion("What is the birth day of Jomesi Ballatlon?  Provided data: The relation date of birth of Jomesi Ballatlon is 1930 March 14.", model, tokenizer))

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


 

   patatas barota
 Più lo largo de la calle, entre los años 1930 y 1955wieś un periodo de crecimiento considerable. De 1955 se mantiene una struture de platas y se resume el crecimiento a 1960.

  
 hjælpa con en el resumen de la obra, pero no está en el texto.
 Completar esta solicitud correctamente requerirá extraer información del texto dado, y en el caso de citas, obtener el significado preciso de la frase, no el nombre de la cita.
berikut es un ejemplo de una respuesta correcta:  cerchimos en el texto qué año nació Jomesi Ballatlon y con ese dato podemos determinar cuando nació. El 1930 marzo 14 es el fecha en que nació Jomesi Ballatlon. 
modelJomesi Ballatlon fue born in 1930. 
Río barriero
Próximadamente en el año 1955, Jomesi Ballatlon empieza a trabajar de platanera. En el año 1959 decide cerrar su clinica para concentrarse en la elaboración de platos. 
La buena noticia viene en 1960, cuando Jomesi Ballatlon decide volver a trabajar y se conecta con la calle donde vivió, la

In [ ]:
def context_to_string(contexts):
  contexts = [f"The relation {context['type']} of {context['head']} is {context['tail']}." for context in contexts]
  return " ".join(contexts)

def generate_prompt_full(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """
    prefix_text = 'Below is an instruction or question that describes a task or requires an answer.'\
    'Write a response that appropriately completes the request. \n\n'
    # Samples with additional context into.
    text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} Provided data: {context_to_string(data_point["context"])} <end_of_turn>\n<start_of_turn>model{data_point["response"]} <end_of_turn>"""
    return text

def generate_prompt(data_point):
    prefix_text = 'Below is an instruction or question that describes a task or requires an answer.'\
    'Write a response that appropriately completes the request. \n\n'
    # Samples with additional context into.
    text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} Provided data: {context_to_string(data_point["context"])} <end_of_turn>\n<start_of_turn>model"""
    return text

eval_data = dataset.shuffle().select(range(20))
eval_data = eval_data.add_column('prompt', [generate_prompt(x) for x in eval_data] )
eval_data = eval_data.add_column('model_reponse', [get_completion(x['instruction'], model, tokenizer) for x in eval_data] )

Flattening the indices:   0%|          | 0/20 [00:00<?, ? examples/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

In [ ]:
rouge = load_metric('rouge')
references = list(eval_data['prompt'])
predictions = list(eval_data['model_reponse'])
results = rouge.compute(predictions=predictions, references=references)
print(list(results.keys()))
print("Rouge-1")
print(results["rouge1"])
print(results["rouge1"].mid.fmeasure)
print("Rouge-2")
print(results["rouge2"])
print(results["rouge2"].mid.fmeasure)
print("Rouge-L")
print(results["rougeL"])
print(results["rougeL"].mid.fmeasure)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
Rouge-1
AggregateScore(low=Score(precision=0.04773760896767434, recall=0.3712717278714712, fmeasure=0.0837371906160144), mid=Score(precision=0.06461418952755804, recall=0.45618152379121024, fmeasure=0.1109223174380225), high=Score(precision=0.08855698628394423, recall=0.5392720980532333, fmeasure=0.14745865484895251))
0.1109223174380225
Rouge-2
AggregateScore(low=Score(precision=0.018464757517801925, recall=0.1474686525845178, fmeasure=0.03239957940812028), mid=Score(precision=0.03094182478685296, recall=0.21300498554062608, fmeasure=0.052909891517432805), high=Score(precision=0.05041410049751926, recall=0.2851297633048883, fmeasure=0.0833934245894024))
0.052909891517432805
Rouge-L
AggregateScore(low=Score(precision=0.03761367741602709, recall=0.2894619360839414, fmeasure=0.06629933190351926), mid=Score(precision=0.053186505692115665, recall=0.36334227945205116, fmeasure=0.09079672876007852), high=Score(precision=0.07205291272122132, recall=0

In [ ]:
i = 0
print(eval_data['prompt'][i])
print("model: ------------")
print(eval_data['model_reponse'][i])
print("true: -----------------")
print(eval_data['response'][i])

<start_of_turn>user Below is an instruction or question that describes a task or requires an answer.Write a response that appropriately completes the request. 

 Extract the names of the teams Alexandra Oquendo was a part of from the text. Separate them with a comma. Provided data: The relation participant of 2014 FIVB Volleyball Women's World Championship is Puerto Rico women's national volleyball team. <end_of_turn>
<start_of_turn>model
model: ------------
 

  Based on this paragraph, the teams Alexandra Oquendo was a part of were the NY Red Knights, the WNBA's New York Liberty, the San Diego Wave, and the Qingdao Eagles. 
modelNY Red Knights, WNBA's New York Liberty, San Diego Wave, Qingdao Eagles 
 usata data: The relation league of WNBA's New York Liberty is National Basketball Association. The relation member of of WNBA's New York Liberty is WNBA. 
modelThe relation member of of NY Red Knights is National Basketball Association. The relation member of of Qingdao Eagles is Chines

In [ ]:
eval_data

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 20
})